In [34]:
import MetaTrader5 as mt5
import pandas as pd
import ta.momentum as momentum
from datetime import datetime
import time
import plotly.graph_objects as go

def get_historical_data(symbol, timeframe, number_of_data=1000):
    if not mt5.initialize():
        print("initialize() failed ☢️")
        mt5.shutdown()
        return None

    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, number_of_data)

    if rates is None:
        print("Failed to retrieve historical data. ☢️")
        mt5.shutdown()
        return None

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")

    mt5.shutdown()

    return df

def calculate_rsi(df, period=14):
    try:
        rsi_indicator = momentum.RSIIndicator(df["close"], window=period)
        df["rsi"] = rsi_indicator.rsi()
    except Exception as e:
        print(f"Error calculating RSI: {e}")

def find_filling_mode(symbol):
    for i in range(2):
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": mt5.symbol_info(symbol).volume_min,
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).ask,
            "type_filling": i,
            "type_time": mt5.ORDER_TIME_GTC
        }

        result = mt5.order_check(request)

        if result.comment == "Done":
            print("Trade was closed")
            break

    return i


In [33]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

def backtest_strategy(df, lot_size=0.2):
    trades = []
    in_trade = False

    for i in range(3, len(df)):
        current_bar = df.iloc[i]
        previous_bar = df.iloc[i - 1]
        confirmation_bar = df.iloc[i - 2]

        # Check for sell conditions
        if current_bar["rsi"] > 70 and confirmation_bar["close"] > confirmation_bar["open"] \
                and previous_bar["close"] < previous_bar["open"] \
                and current_bar["close"] < current_bar["open"]:
            
            if not in_trade:
                # Open a sell trade
                trade = {"entry_time": current_bar.name, "entry_price": current_bar["open"], "trade_type": "SELL"}
                trades.append(trade)
                in_trade = True

        # Check for trade closing condition
        if in_trade:
            if current_bar["close"] < current_bar["open"]:
                # Close the trade
                trades[-1]["exit_time"] = current_bar.name
                trades[-1]["exit_price"] = current_bar["close"]
                trades[-1]["pnl"] = trades[-1]["entry_price"] - trades[-1]["exit_price"]
                trades[-1]["pnl"] *= lot_size
                in_trade = False

    return trades



def plot_trades(df, trades):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df['open'],
                                        high=df['high'],
                                        low=df['low'],
                                        close=df['close'])])

    for trade in trades:
        entry_time = trade["entry_time"]
        entry_price = trade["entry_price"]
        exit_time = trade["exit_time"]
        exit_price = trade["exit_price"]

        fig.add_trace(go.Scatter(x=[entry_time, exit_time],
        y=[entry_price, exit_price],
        mode='markers',
        marker=dict(size=[10, 10],
        symbol=['triangle-up', 'triangle-down'],
        color=['green', 'red'],
        line=dict(width=2))))

    fig.update_layout(title="Backtest Trades",
    xaxis_title="Time",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False)
    
    fig.show()

if __name__ == "__main__":
    symbol = "Boom 1000 Index"
    timeframe = mt5.TIMEFRAME_M1
    lot_size = 1.5
    data_length = 1000
    period = 14

    # Step 1: Load historical data
    df = get_historical_data(symbol, timeframe, data_length)

    # Step 2: Calculate RSI
    calculate_rsi(df, period)

    # Step 3: Backtest the strategy
    trades = backtest_strategy(df, lot_size)

    # Step 4: Print the backtest results
    total_pnl = 0
    for trade in trades:
        print(f"Trade Type: {trade['trade_type']}")
        print(f"Entry Time: {trade['entry_time']}, Entry Price: {trade['entry_price']}")
        print(f"Exit Time: {trade['exit_time']}, Exit Price: {trade['exit_price']}")
        print(f"Profit/Loss: {trade['pnl']:.2f}\n")
        total_pnl += trade['pnl']

    print(f"Total Profit/Loss: {total_pnl:.2f}")

    # Step 5: Plot the trades
    plot_trades(df, trades)


Trade Type: SELL
Entry Time: 2023-08-06 21:54:00, Entry Price: 11142.884
Exit Time: 2023-08-06 21:54:00, Exit Price: 11142.277
Profit/Loss: 0.91

Trade Type: SELL
Entry Time: 2023-08-06 22:08:00, Entry Price: 11156.499
Exit Time: 2023-08-06 22:08:00, Exit Price: 11155.881
Profit/Loss: 0.93

Trade Type: SELL
Entry Time: 2023-08-06 23:17:00, Entry Price: 11145.263
Exit Time: 2023-08-06 23:17:00, Exit Price: 11144.67
Profit/Loss: 0.89

Trade Type: SELL
Entry Time: 2023-08-06 23:21:00, Entry Price: 11151.917
Exit Time: 2023-08-06 23:21:00, Exit Price: 11151.26
Profit/Loss: 0.99

Trade Type: SELL
Entry Time: 2023-08-06 23:58:00, Entry Price: 11169.897
Exit Time: 2023-08-06 23:58:00, Exit Price: 11169.188
Profit/Loss: 1.06

Trade Type: SELL
Entry Time: 2023-08-07 00:06:00, Entry Price: 11185.073
Exit Time: 2023-08-07 00:06:00, Exit Price: 11184.377
Profit/Loss: 1.04

Trade Type: SELL
Entry Time: 2023-08-07 01:59:00, Entry Price: 11134.03
Exit Time: 2023-08-07 01:59:00, Exit Price: 11133.289
